In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch

torch.cuda.get_device_name()

'Tesla P40'

In [2]:
import random
from datasets import load_dataset
from transformers import pipeline

In [3]:
# Load dataset
dataset_name = "locchh/nvidia_qa"
dataset = load_dataset(dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 9082
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 1135
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1136
    })
})